[View in Colaboratory](https://colab.research.google.com/github/Zonglin-Li6565/smiley-face/blob/master/facekoob_new.ipynb)

In [0]:
!/opt/bin/nvidia-smi

In [0]:
!free -m

In [0]:
try:
  import dlib
except ImportError as error:
  !apt update
  !apt install -y cmake
  !pip install dlib
  import dlib

In [0]:
import argparse
import glob
import logging
import os
import pickle
import random
import sys
import time

import numpy as np

import cv2

from google.colab import files

In [0]:
try:
  import torch
except ImportError as error:
  from os import path
  from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
  platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
  accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

  !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision
  import torch

In [0]:
import torch
import torch.multiprocessing as mp
from torch import optim
import torch.nn as nn
from torch.autograd import Variable
from torch.nn import Module, functional

# Get the dataset and opencv classifier xml

In [0]:
if not os.path.isfile('lfw.tgz'):
  !wget http://vis-www.cs.umass.edu/lfw/lfw.tgz
  !tar xzf lfw.tgz
  !wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
  !bzip2 -d shape_predictor_68_face_landmarks.dat.bz2

In [0]:
!ls

In [0]:
!lscpu

#Preprocess the faces
Will find the face, crop it and set a file flag when done

In [0]:
# Copied from https://github.com/cmusatyalab/openface/blob/master/openface/align_dlib.py
TEMPLATE = np.float32([
    (0.0792396913815, 0.339223741112), (0.0829219487236, 0.456955367943),
    (0.0967927109165, 0.575648016728), (0.122141515615, 0.691921601066),
    (0.168687863544, 0.800341263616), (0.239789390707, 0.895732504778),
    (0.325662452515, 0.977068762493), (0.422318282013, 1.04329000149),
    (0.531777802068, 1.06080371126), (0.641296298053, 1.03981924107),
    (0.738105872266, 0.972268833998), (0.824444363295, 0.889624082279),
    (0.894792677532, 0.792494155836), (0.939395486253, 0.681546643421),
    (0.96111933829, 0.562238253072), (0.970579841181, 0.441758925744),
    (0.971193274221, 0.322118743967), (0.163846223133, 0.249151738053),
    (0.21780354657, 0.204255863861), (0.291299351124, 0.192367318323),
    (0.367460241458, 0.203582210627), (0.4392945113, 0.233135599851),
    (0.586445962425, 0.228141644834), (0.660152671635, 0.195923841854),
    (0.737466449096, 0.182360984545), (0.813236546239, 0.192828009114),
    (0.8707571886, 0.235293377042), (0.51534533827, 0.31863546193),
    (0.516221448289, 0.396200446263), (0.517118861835, 0.473797687758),
    (0.51816430343, 0.553157797772), (0.433701156035, 0.604054457668),
    (0.475501237769, 0.62076344024), (0.520712933176, 0.634268222208),
    (0.565874114041, 0.618796581487), (0.607054002672, 0.60157671656),
    (0.252418718401, 0.331052263829), (0.298663015648, 0.302646354002),
    (0.355749724218, 0.303020650651), (0.403718978315, 0.33867711083),
    (0.352507175597, 0.349987615384), (0.296791759886, 0.350478978225),
    (0.631326076346, 0.334136672344), (0.679073381078, 0.29645404267),
    (0.73597236153, 0.294721285802), (0.782865376271, 0.321305281656),
    (0.740312274764, 0.341849376713), (0.68499850091, 0.343734332172),
    (0.353167761422, 0.746189164237), (0.414587777921, 0.719053835073),
    (0.477677654595, 0.706835892494), (0.522732900812, 0.717092275768),
    (0.569832064287, 0.705414478982), (0.635195811927, 0.71565572516),
    (0.69951672331, 0.739419187253), (0.639447159575, 0.805236879972),
    (0.576410514055, 0.835436670169), (0.525398405766, 0.841706377792),
    (0.47641545769, 0.837505914975), (0.41379548902, 0.810045601727),
    (0.380084785646, 0.749979603086), (0.477955996282, 0.74513234612),
    (0.523389793327, 0.748924302636), (0.571057789237, 0.74332894691),
    (0.672409137852, 0.744177032192), (0.572539621444, 0.776609286626),
    (0.5240106503, 0.783370783245), (0.477561227414, 0.778476346951)])

INV_TEMPLATE = np.float32([
    (-0.04099179660567834, -0.008425234314031194, 2.575498465013183),
    (0.04062510634554352, -0.009678089746831375, -1.2534351452524177),
    (0.0003666902601348179, 0.01810332406086298, -0.32206331976076663)])

In [0]:
TPL_MIN, TPL_MAX = np.min(TEMPLATE, axis=0), np.max(TEMPLATE, axis=0)
MINMAX_TEMPLATE = (TEMPLATE - TPL_MIN) / (TPL_MAX - TPL_MIN)

In [0]:
INNER_EYES_AND_BOTTOM_LIP = [39, 42, 57]
OUTER_EYES_AND_NOSE = [36, 45, 33]

In [0]:
class CropAndAlign:
    def __init__(self, face_predictor_path, landmark_indices, logger):
        self.detector = dlib.get_frontal_face_detector()
        self.land_mark_predictor = dlib.shape_predictor(face_predictor_path)
        self.land_mark_indices = np.array(landmark_indices)
        self.logger = logger

    def find_all_bounding_boxes(self, rgb_img):
        try:
            # Upsample the image once
            return self.detector(rgb_img, 1)
        except Exception as e:
            self.logger.warn(e)
            return []

    def get_largest_bounding_box(self, rgb_img):
        faces = self.find_all_bounding_boxes(rgb_img)
        if len(faces) > 0:
            return max(faces, key=lambda box: box.width() * box.height())
        else:
            self.logger.warn('No face was found in image')
            return None

    def find_landmarks(self, rgb_img, box):
        points = self.land_mark_predictor(rgb_img, box)
        return np.float32(list(map(lambda p: (p.x, p.y), points.parts())))

    def align_one_face(self, rgb_img, box, out_size):
        landmarks = self.find_landmarks(rgb_img, box)
        affine_transform = cv2.getAffineTransform(
            landmarks[self.land_mark_indices],
            out_size * MINMAX_TEMPLATE[self.land_mark_indices])
        return cv2.warpAffine(rgb_img, affine_transform, (out_size, out_size))

    def align_biggest_face(self, rgb_img, out_size):
        box = self.get_largest_bounding_box(rgb_img)
        if box is not None:
            return self.align_one_face(rgb_img, box, out_size)
        else:
            return None

    def align_all_faces(self, rgb_img, out_size):
        boxes = self.find_all_bounding_boxes(rgb_img)
        return [self.align_one_face(rgb_img, box, out_size) for box in boxes]

In [0]:
def process_image_(aligner, in_path, out_path, dim):
    logger.debug('Processing %s' % in_path)
    image = cv2.imread(in_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    if image is None:
        logger.error('Failed to load image %s' % in_path)
        return
    aligned = aligner.align_biggest_face(image, dim)
    if aligned is None:
        logger.warning('No face found in %s' % in_path)
    else:
        aligned = cv2.cvtColor(aligned, cv2.COLOR_BGR2RGB)
        cv2.imwrite(out_path, aligned)
        del aligned

In [0]:
def preprocess_dataset(input_dir,
                       output_dir,
                       out_dim,
                       face_predictor_path,
                       logger,
                       landmark_indices=INNER_EYES_AND_BOTTOM_LIP):
    if os.path.exists(output_dir):
        logger.error('Output dir %s exists' % output_dir)
        return
    if not os.path.exists(input_dir):
        logger.error('Data dir %s doesn\'t exist' % input_dir)
        return
    elif not os.path.exists(face_predictor_path):
        logger.error('Predictor %s doesn\'t exist' % face_predictor_path)
        return

    aligner = CropAndAlign(face_predictor_path, landmark_indices, logger)

    global global_aligner
    global_aligner = aligner

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for image_dir in os.listdir(input_dir):
        image_output_dir = os.path.join(
            output_dir, os.path.basename(os.path.basename(image_dir)))
        if not os.path.exists(image_output_dir):
            os.makedirs(image_output_dir)

    image_paths = glob.glob(os.path.join(input_dir, '**/*.jpg'))

    for image_path in image_paths:
        image_output_dir = os.path.join(
            output_dir, os.path.basename(os.path.dirname(image_path)))
        output_path = os.path.join(image_output_dir,
                                   os.path.basename(image_path))
        process_image_(aligner, image_path, output_path, out_dim)

In [0]:
logging.basicConfig(level=logging.INFO, 
                    format='%(asctime)s;%(levelname)s;%(message)s')
logger = logging.getLogger(__name__)
preprocess_dataset('lfw', 'processed', 72, 
                   'shape_predictor_68_face_landmarks.dat', logger)


# Define the model

In [0]:
class Embedder(Module):
    def __init__(self, input_size, kernel_sizes):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=kernel_sizes[0])
        self.pool1 = nn.MaxPool2d(kernel_size=kernel_sizes[1], stride=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=kernel_sizes[2])
        self.pool2 = nn.MaxPool2d(kernel_size=kernel_sizes[3], stride=1)
        self.conv3 = nn.Conv2d(128, 64, kernel_size=kernel_sizes[4])
        self.pool3 = nn.AvgPool2d(kernel_size=kernel_sizes[5], stride=1)
        self.conv4 = nn.Conv2d(64, 32, kernel_size=kernel_sizes[6])
        self.pool4 = nn.AvgPool2d(kernel_size=kernel_sizes[7], stride=1)
        size_reduction = sum(kernel_sizes) - len(kernel_sizes)
        self.fc_input_dimension = ((input_size[0] - size_reduction) *
                                   (input_size[1] - size_reduction) * 32)
        self.fc1 = nn.Linear(self.fc_input_dimension, 256)
        self.fc2 = nn.Linear(256, 128)

    def forward(self, x):
        x = Variable(x, requires_grad=False)
        x = functional.relu(self.pool1(self.conv1(x)))
        x = functional.dropout2d(x, p=0.5)
        x = functional.relu(self.pool2(self.conv2(x)))
        x = functional.dropout2d(x, p=0.2)
        x = functional.relu(self.pool3(self.conv3(x)))
        x = functional.dropout2d(x, p=0.1)
        x = functional.relu(self.pool4(self.conv4(x)))
        x = x.view(-1, self.fc_input_dimension)
        x = functional.relu(self.fc1(x))
        x = functional.sigmoid(self.fc2(x))
        return x

# Train the model

In [0]:
!ls

In [0]:
class DataGenerator:
    def __init__(self, data_dir, batch_size, logger):
        self.data_dir = data_dir
        self.face_images = []
        if not os.path.exists(data_dir):
            logger.error('Data dir %s doesn\'t exist' % data_dir)
            return
        people = os.listdir(data_dir)
        for person in people:
            person_dir = os.path.join(data_dir, person)
            faces = os.listdir(person_dir)
            if len(faces) > 0:
                face_images = list(
                    map(lambda img_name: os.path.join(person_dir, img_name),
                        faces))
                self.face_images.append(face_images)
        self.batch_size = batch_size
        self.logger = logger
        self.person_idx = 0
        self.image_idx = 0

    def __iter__(self):
        return self

    def __next__(self):
        anchors = []
        positives = []
        negatives = []

        batch_counter = 0
        while (batch_counter < self.batch_size) and len(self.face_images) > 1:
            if self.person_idx == 0:
                random.shuffle(self.face_images)
                self.person_idx += 1
            if self.image_idx == 0:
                random.shuffle(self.face_images[self.person_idx])
                self.image_idx = 1

            if len(self.face_images[self.person_idx]) > 1:
                anchors.append(
                    cv2.imread(self.face_images[self.person_idx][
                        self.image_idx]).astype(np.float))
                positives.append(
                    cv2.imread(self.face_images[self.person_idx][self.image_idx
                                                                 - 1]).astype(
                                                                     np.float))
                negative_idx = random.randint(
                    0,
                    len(self.face_images[self.person_idx - 1]) - 1)
                negatives.append(
                    cv2.imread(self.face_images[self.person_idx -
                                                1][negative_idx]).astype(
                                                    np.float))
                batch_counter += 1

            self.image_idx += 1
            if self.image_idx >= len(self.face_images[self.person_idx]):
                self.image_idx = 0
                self.person_idx += 1
                self.person_idx %= len(self.face_images)

        anchors = torch.Tensor(anchors)
        positives = torch.Tensor(positives)
        negatives = torch.Tensor(negatives)
        batch = torch.cat((anchors, positives, negatives), dim=0)
        batch = torch.transpose(batch, 1, 3)
        batch = torch.transpose(batch, 2, 3)
        return batch

In [0]:
class LoadingWorker(mp.Process):
    def __init__(self, data_dir, batch_size, queue):
        super().__init__()
        logging.basicConfig(level=logging.INFO)
        self.logger = logging.getLogger(__name__)
        self.generator = DataGenerator(data_dir, batch_size, self.logger)
        self.queue = queue
        self.exit = mp.Event()

    def run(self):
        while not self.exit.is_set():
            batch = self.generator.__next__()
            self.queue.put(batch)
        time.sleep(2)

    def terminate(self):
        self.logger.info('Shutting down loader')
        self.exit.set()
        while not self.queue.empty():
            self.queue.get()

In [0]:
def loss_fn(batch_size, embeddings, alpha=0.2):
    anchor = embeddings[:batch_size]
    positive = embeddings[batch_size:2 * batch_size]
    negative = embeddings[2 * batch_size:]
    loss = (
        (torch.norm(anchor - positive)**2 - torch.norm(anchor - negative)**2) /
        batch_size + alpha)
    return loss

In [0]:
def train(data_dir,
          batch_size,
          total_iter,
          kernel_sizes,
          logger,
          model_save_path,
          input_shape=(72, 72),
          cuda=True):
    queue = mp.Queue(10)
    process = LoadingWorker(data_dir, batch_size, queue)
    process.start()
    model = Embedder(input_shape, kernel_sizes)
    if cuda:
        model = model.cuda()
    optimizer = optim.Adam(model.parameters(), lr=5e-6)
    for i in range(total_iter):
        optimizer.zero_grad()
        images = queue.get()
        if cuda:
            images = images.cuda()
        embeddings = model(images)
        loss = loss_fn(batch_size, embeddings)
        loss.backward()
        optimizer.step()
        if i % 50 == 0:
            logger.info('loss at step %04d is: %f' % (i, loss))
    process.terminate()
    process.join()
    torch.save(model.cpu().state_dict(), model_save_path)

In [0]:
DATA_DIR = 'processed/'
BATCH_SIZE = 32
TOTAL_ITER = 40000
KERNEL_SIZES = [5, 3, 5, 3, 3, 3, 3, 3]
train(DATA_DIR, BATCH_SIZE, TOTAL_ITER, KERNEL_SIZES, logger,
      'model-weights')

# Download the model

In [0]:
files.download('model-weights')

In [0]:
!ls -l